In [23]:
import os
from fancy import Data, Model, Analysis

In [24]:
'''Setting up'''
# Define location of Stan files
stan_path = '../../../stan/'

# Define file containing source catalogue information
source_file = '../../../data/sourcedata.h5'
uhecr_file = '../../../data/UHECRdata.h5'

# make output directory if it doesnt exist
if not os.path.isdir("output"):
    os.mkdir("output")

# source_types = ["SBG_23", "2FHL_250Mpc", "swift_BAT_213"]
source_types = ["SBG_23"]

# detector_types = ["auger2010", "auger2014", "TA2015"]
# detector_type = "auger2014"
detector_type = "TA2015"

# set random seed
random_seed = 19990308

# flag to control showing plots or not
show_plot = True

In [25]:
'''set detector and detector properties'''
if detector_type == "TA2015":
    from fancy.detector.TA2015 import detector_properties, alpha_T, M, Eth
elif detector_type == "auger2014":
    from fancy.detector.auger2014 import detector_properties, alpha_T, M, Eth
elif detector_type == "auger2010":
    from fancy.detector.auger2010 import detector_properties, alpha_T, M, Eth
else:
    raise Exception("Undefined detector type!")

In [26]:
'''Fit arrival model'''
for source_type in source_types:
    # table file
    table_file = '../../../tables/tables_{0}_{1}.h5'.format(source_type, detector_type)    
    # define output files
    arrival_output_file = 'output/arrival_fit_{0}_{1}.h5'.format(source_type, detector_type)

    # construct Dataset
    data = Data()
    data.add_source(source_file, source_type)
    data.add_uhecr(uhecr_file, detector_type, ptype="He")
    data.add_detector(detector_properties)


    # data.show();

    # construct arrival model obejct
    arrival_model = stan_path + 'arrival_direction_model.stan'
    model = Model(model_filename = arrival_model, include_paths = stan_path)
    model.compile()
    model.input(Eth = Eth) # EeV

    # What is happening 
    # summary = b'Fit of the joint model to the Auger data' 
    summary = b'Fit of the arrival direction model to data' 
        
    # Define an Analysis object to bring together Data and Model objects
    analysis = Analysis(data, model, analysis_type = 'joint', 
                        filename = arrival_output_file, summary = summary)

    # Define location of pre-computed values used in fits 
    # (see relevant notebook for how to make these files) 
    # Each catalogue has a file of pre-computed values
    analysis.use_tables(table_file)

    # Fit the Stan model
    fit = analysis.fit_model(chains = 16, iterations = 500, seed = random_seed)

    # Save to analysis file
    analysis.save()

Using cached StanModel
Performing fitting...


Checking all diagnostics...
Done!


In [27]:
'''Fit joint model'''
for source_type in source_types:
    # table file
    table_file = '../../../tables/tables_{0}_{1}.h5'.format(source_type, detector_type)    
    # define output files
    joint_output_file = 'output/joint_fit_{0}_{1}.h5'.format(source_type, detector_type)

    # construct Dataset
    data = Data()
    data.add_source(source_file, source_type)
    data.add_uhecr(uhecr_file, detector_type)
    data.add_detector(detector_properties)

    # if show_plot:
    #     data.show()

    joint_model = stan_path + 'joint_model_tightB.stan'
    model = Model(model_filename = joint_model, include_paths = stan_path)
    model.compile()
    model.input(Eth = Eth) # EeV

    # What is happening 
    # summary = b'Fit of the joint model to the Auger data' 
    summary = b'Fit of the joint model to data' 
        
    # Define an Analysis object to bring together Data and Model objects
    analysis = Analysis(data, model, analysis_type = 'joint', 
                        filename = joint_output_file, summary = summary)

    # Define location of pre-computed values used in fits 
    # (see relevant notebook for how to make these files) 
    # Each catalogue has a file of pre-computed values
    analysis.use_tables(table_file)

    # Fit the Stan model
    fit = analysis.fit_model(chains = 16, iterations = 500, seed = random_seed)

    # Save to analysis file
    analysis.save()

Using cached StanModel
Performing fitting...


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Checking all diagnostics...
Done!


In [29]:
'''Fit joint + gmf model'''

ptype = "Fe"
for source_type in source_types:
    # table file
    table_file = '../../../tables/tables_{0}_{1}.h5'.format(source_type, detector_type)    
    # define output files
    joint_output_file = 'output/joint_gmf_fit_{0}_{1}_{2}.h5'.format(source_type, detector_type, ptype)

    # construct Dataset
    data = Data()
    data.add_source(source_file, source_type)
    data.add_uhecr(uhecr_file, detector_type, ptype=ptype)
    data.add_detector(detector_properties)

    # if show_plot:
    #     data.show()

    joint_model = stan_path + 'joint_gmf_model_tightB.stan'
    model = Model(model_filename = joint_model, include_paths = stan_path)
    model.compile()
    model.input(Eth = Eth) # EeV

    # What is happening 
    # summary = b'Fit of the joint model to the Auger data' 
    summary = b'Fit of the joint model to data' 
        
    # Define an Analysis object to bring together Data and Model objects
    analysis = Analysis(data, model, analysis_type = 'joint_gmf', 
                        filename = joint_output_file, summary = summary)

    # Define location of pre-computed values used in fits 
    # (see relevant notebook for how to make these files) 
    # Each catalogue has a file of pre-computed values
    analysis.use_tables(table_file)

    # Fit the Stan model
    fit = analysis.fit_model(chains = 16, iterations = 500, seed = random_seed)

    # Save to analysis file
    analysis.save()

Using cached StanModel
Performing fitting...


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Checking all diagnostics...
Done!
